In [11]:
def pri(A):
    for i in A:
        print(i)

In [12]:
def andFunc(a,b):
    if(a==0 or b==0):
        return 0
    else:
        return 1

def orFunc(a,b):
    if(a==1 or b==1):
        return 1
    else:
        return 0

def invFunc(a):
    if(a==1):
        return 0
    else:
        return 1

def norFunc(a,b):
    if(a==1 or b==1):
        return 0
    else:
        return 1

def nandFunc(a,b):
    if(a==0 or b==0):
        return 1
    else:
        return 0

def xorFunc(a,b):
    if ((a==1 and b==0) or (a==0 and b==1)):
        return 1
    else:
        return 0
        
def xnorFunc(a,b):
    if ((a==1 and b==0) or (a==0 and b==1)):
        return 0
    else:
        return 1


# TOPOLOGICAL SORT FOR CIRCUIT EVALUATION

In [13]:
import networkx as nx
import matplotlib.pyplot as plt

## Reading the netlist

In [14]:
# Reading the data file
def readNetlist(points):
    data = []
    for point in points:
        point = point.rstrip('\n') 
        point = point.split()
        data.append(point)
    return data 

with open('./benchmarks/c17.net') as f:
    lines = f.readlines()

netList = readNetlist(lines)
pri(netList)

['g51', 'nand2', 'n_3', 'n_0', 'N22']
['g52', 'nand2', 'n_3', 'n_2', 'N23']
['g53', 'nand2', 'n_1', 'N2', 'n_3']
['g54', 'nand2', 'n_1', 'N7', 'n_2']
['g55', 'nand2', 'N1', 'N3', 'n_0']
['g56', 'nand2', 'N3', 'N6', 'n_1']


## Reading the inputs

In [15]:
# Reading the input file
def readInput(points):
    data = []
    for point in points:
        point = point.rstrip('\n') 
        point = point.split()
        data.append(point)
    return data 

with open('./benchmarks/c17.inputs') as f:
    ln = f.readlines()

netInput = readNetlist(ln)
pri(netInput)

['N1', 'N2', 'N3', 'N6', 'N7']
['0', '1', '0', '0', '0']
['0', '0', '1', '0', '0']
['1', '0', '0', '0', '0']
['0', '0', '1', '1', '1']
['1', '1', '1', '1', '1']
['1', '1', '1', '0', '0']
['1', '1', '1', '1', '0']
['1', '1', '0', '0', '0']
['0', '1', '1', '0', '1']
['0', '0', '1', '1', '0']


In [16]:
g = nx.DiGraph()

for i in netList:
    print(i)
    if(i[1]=='inv'):
        g.add_edge(i[2],i[3])
    else:
        g.add_edge(i[2],i[4])
        g.add_edge(i[3],i[4])
# pri(g)
nl =  list(nx.topological_sort(g))
print('Nodes in topological order', nl)



['g51', 'nand2', 'n_3', 'n_0', 'N22']
['g52', 'nand2', 'n_3', 'n_2', 'N23']
['g53', 'nand2', 'n_1', 'N2', 'n_3']
['g54', 'nand2', 'n_1', 'N7', 'n_2']
['g55', 'nand2', 'N1', 'N3', 'n_0']
['g56', 'nand2', 'N3', 'N6', 'n_1']
Nodes in topological order ['N2', 'N7', 'N1', 'N3', 'N6', 'n_0', 'n_1', 'n_3', 'n_2', 'N22', 'N23']


In [17]:
info = {}
for i in netList:
    if(i[1]=='inv'):
        info.update({i[3]:[i[1],i[2]]})
    else:
        info.update({i[4]:[i[1],i[2],i[3]]})

print(info)


{'N22': ['nand2', 'n_3', 'n_0'], 'N23': ['nand2', 'n_3', 'n_2'], 'n_3': ['nand2', 'n_1', 'N2'], 'n_2': ['nand2', 'n_1', 'N7'], 'n_0': ['nand2', 'N1', 'N3'], 'n_1': ['nand2', 'N3', 'N6']}


In [18]:
inputDict = {}

for i in range (len(netInput[0])):
    inputDict.update({netInput[0][i]:[]})

for i in range(1, len(netInput)):
    for j in range(len(netInput[0])):
        inputDict[netInput[0][j]].append(int(netInput[i][j]))

print(inputDict)

{'N1': [0, 0, 1, 0, 1, 1, 1, 1, 0, 0], 'N2': [1, 0, 0, 0, 1, 1, 1, 1, 1, 0], 'N3': [0, 1, 0, 1, 1, 1, 1, 0, 1, 1], 'N6': [0, 0, 0, 1, 1, 0, 1, 0, 0, 1], 'N7': [0, 0, 0, 1, 1, 0, 0, 0, 1, 0]}


In [19]:
# , len(netInput)
print(nl)
for i in range (1, len(netInput)):
    output = []
    outputDict = {}

    for j in range (len(nl)):
        output.append(None)
    
    # print(output)
    
    for j in range (len(nl)):
        if(nl[j] in inputDict.keys()):
            output[j] = inputDict[nl[j]][i-1]
            outputDict.update({nl[j]:output[j]})
            # print(nl[j]," ",inputDict[nl[j]][i-1], " ",output[j])

        else:
            if (info[nl[j]][0]=='and2'):
                output[j] = andFunc(outputDict[info[nl[j]][1]],outputDict[info[nl[j]][2]])
                outputDict.update({nl[j]:output[j]})

            elif(info[nl[j]][0]=='or2'):
                output[j] = orFunc(outputDict[info[nl[j]][1]],outputDict[info[nl[j]][2]])
                outputDict.update({nl[j]:output[j]})

            elif (info[nl[j]][0]=='inv'):
                output[j] = invFunc(outputDict[info[nl[j]][1]])
                outputDict.update({nl[j]:output[j]})

            elif (info[nl[j]][0]=='nor2'):
                output[j] = norFunc(outputDict[info[nl[j]][1]],outputDict[info[nl[j]][2]])
                outputDict.update({nl[j]:output[j]})

            elif (info[nl[j]][0]=='nand2'):
                output[j] = nandFunc(outputDict[info[nl[j]][1]],outputDict[info[nl[j]][2]])
                outputDict.update({nl[j]:output[j]})

            elif (info[nl[j]][0]=='xor2'):
                output[j] = xorFunc(outputDict[info[nl[j]][1]],outputDict[info[nl[j]][2]])
                outputDict.update({nl[j]:output[j]})
                
            elif (info[nl[j]][0]=='xnor2'):
                output[j] = xnorFunc(outputDict[info[nl[j]][1]],outputDict[info[nl[j]][2]])
                outputDict.update({nl[j]:output[j]})
                
    print((output))
            



['N2', 'N7', 'N1', 'N3', 'N6', 'n_0', 'n_1', 'n_3', 'n_2', 'N22', 'N23']
[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1]
[0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0]
[1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0]
[1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1]
[1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1]
[0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0]
